In [1]:
import platform; print(platform.platform())
import sys; print("Python", sys.version)

Windows-10-10.0.19045-SP0
Python 3.10.5 (tags/v3.10.5:f377153, Jun  6 2022, 16:14:13) [MSC v.1929 64 bit (AMD64)]


In [2]:
import numpy as np
import pandas as pd

import os 
import joblib
import time

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error

import optuna
import lightgbm as lgb

In [3]:
class cfg:
    modelname = 'lgb'
    debug = False
    optim = True
    seed = 42
    nfolds = 5
    njobs = 2

In [4]:
# load data
train = pd.read_csv('../data/final/train.csv')
test = pd.read_csv('../data/final/test.csv')

train.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal,IsSynthetic,...,Monterey County,Napa County,Orange County,Other,Riverside County,San Francisco County,Santa Barbara County,Santa Clara County,Ventura County,Yolo County
0,2.3859,15.0,3.827160,1.112100,1280.0,2.486989,34.60,-120.12,0.980,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,3.7188,17.0,6.013373,1.054217,1504.0,3.813084,38.69,-121.22,0.946,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.7750,27.0,6.535604,1.103175,1061.0,2.464602,34.71,-120.45,1.576,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2.4138,16.0,3.350203,0.965432,1255.0,2.089286,32.66,-117.09,1.336,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.7500,52.0,4.284404,1.069246,1793.0,1.604790,37.80,-122.41,4.500,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [5]:
# quick info
TARGET = 'MedHouseVal'
FEATURES = [c for c in train.columns if c not in [TARGET]]

print(f'Target: {TARGET}\nFeatures: {FEATURES}')
print('Train set shape:', train.shape)
print('Test set shape:', test.shape)

x = train[FEATURES]
y = train[TARGET]

Target: MedHouseVal
Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude', 'IsSynthetic', 'AnomalyScore', 'rot_15_x', 'rot_15_y', 'rot_30_x', 'rot_30_y', 'rot_45_x', 'pca_lon', 'pca_lat', 'Alameda County', 'Contra Costa County', 'Fresno County', 'Kern County', 'Los Angeles County', 'Monterey County', 'Napa County', 'Orange County', 'Other', 'Riverside County', 'San Francisco County', 'Santa Barbara County', 'Santa Clara County', 'Ventura County', 'Yolo County']
Train set shape: (57777, 33)
Test set shape: (24759, 32)


In [6]:
# setup
params_path = './training_files/params/'
os.makedirs(params_path, exist_ok=True)

cv = KFold(n_splits=cfg.nfolds, shuffle=True, random_state=cfg.seed)

In [7]:
# fixed params
fixed_params = {
        'objective': 'regression',
        'metric': 'rmse',
        'device': 'cpu',
        'verbosity': -1,
        'early_stopping_round': 15,
        'boosting_type': 'gbdt' 
    }

# objective function for optimization
def objective(trial):
    
    # trial parameters
    tuning_params = {
        #'boosting_type': trial.suggest_categorical('boosting', ['gbdt', 'dart']),
        'n_estimators': trial.suggest_int('n_estimators', 100, 2500),
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.3),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    params = {**fixed_params, **tuning_params}

    # train and score with cv
    scores = []
    for train_idx, test_idx in cv.split(x, y):
        
        # split data
        x_train, x_val = x.iloc[train_idx], x.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]
        
        # fit model
        model = lgb.LGBMRegressor(**params)
        model.fit(
            x_train,
            y_train,
            eval_set=[(x_val, y_val)],
            callbacks=[lgb.log_evaluation(period=0, show_stdv=False)]
        )
        
        # obtain score
        rmse = np.sqrt(mean_squared_error(y_val, model.predict(x_val)))
        scores.append(rmse)

    # return mean cv score 
    return np.mean(scores)

In [8]:
if cfg.optim:

    # create study
    sampler = optuna.samplers.TPESampler(seed=cfg.seed)
    max_trials = 2 if cfg.debug else 100
    time_limit = 3600 * 0.5

    study = optuna.create_study(
        sampler=sampler,
        study_name= f'{cfg.modelname}_optimization',
        direction='minimize')

    # perform optimization
    print(f'Starting {cfg.modelname} optimization...')
    study.optimize(
        objective,
        n_trials = max_trials,
        timeout = time_limit,
    )

[I 2023-01-07 16:05:10,385] A new study created in memory with name: lgb_optimization


Starting lgb optimization...


[I 2023-01-07 16:05:12,805] Trial 0 finished with value: 0.5318930920830947 and parameters: {'n_estimators': 999, 'learning_rate': 0.28767857660247903, 'reg_alpha': 7.31993942079411, 'reg_lambda': 5.9865848459837805, 'num_leaves': 81, 'colsample_bytree': 0.49359671220172163, 'subsample': 0.4348501673009197, 'subsample_freq': 7, 'min_child_samples': 62}. Best is trial 0 with value: 0.5318930920830947.
[I 2023-01-07 16:05:25,690] Trial 1 finished with value: 0.5158469735649029 and parameters: {'n_estimators': 1800, 'learning_rate': 0.055146123573950614, 'reg_alpha': 9.699098521920845, 'reg_lambda': 8.324426409679791, 'num_leaves': 110, 'colsample_bytree': 0.5090949803242604, 'subsample': 0.5100427059120604, 'subsample_freq': 3, 'min_child_samples': 55}. Best is trial 1 with value: 0.5158469735649029.


In [9]:
if cfg.optim:
    
    # optimization results
    print(f"Number of finished trials: {len(study.trials)}")
    print(f"Best score: {study.best_value}")
    best_params = {**fixed_params, **study.best_trial.params}
    print("Best trial parameters:")
    for k, v in best_params.items():
        print(f"\t{k}: {v}")

    # save best params
    best_params_path = f'{params_path}{cfg.modelname}.joblib'
    with open(best_params_path, "wb") as file:
        joblib.dump(best_params, file)

Number of finished trials: 2
Best score: 0.5158469735649029
Best trial parameters:
	objective: regression
	metric: rmse
	device: cpu
	verbosity: -1
	early_stopping_round: 15
	boosting_type: gbdt
	n_estimators: 1800
	learning_rate: 0.055146123573950614
	reg_alpha: 9.699098521920845
	reg_lambda: 8.324426409679791
	num_leaves: 110
	colsample_bytree: 0.5090949803242604
	subsample: 0.5100427059120604
	subsample_freq: 3
	min_child_samples: 55


In [10]:
# load best params
with open(best_params_path, 'rb') as file:
    best_params = joblib.load(file)

print("Final parameters:")
for k, v in best_params.items():
    print(f"\t{k}: {v}")

Final parameters:
	objective: regression
	metric: rmse
	device: cpu
	verbosity: -1
	early_stopping_round: 15
	boosting_type: gbdt
	n_estimators: 1800
	learning_rate: 0.055146123573950614
	reg_alpha: 9.699098521920845
	reg_lambda: 8.324426409679791
	num_leaves: 110
	colsample_bytree: 0.5090949803242604
	subsample: 0.5100427059120604
	subsample_freq: 3
	min_child_samples: 55


In [11]:
# # increase cv folds for final prediction
# cfg.nfolds = 10

In [12]:
# cv training and predict
def train_model(train, test, params):

    start_time = time.time()
    print(30*'*', f'Model: {cfg.modelname}', 30*'*', '\n')

    # get data
    x = train[FEATURES]
    y = train[TARGET]
    xtest = test[FEATURES]

    # cv loop
    cv = KFold(n_splits=cfg.nfolds, shuffle=True, random_state=cfg.seed)
    preds, cv_scores = [], []
    for fold, (train_idx, test_idx) in enumerate(cv.split(x, y)):

        print(f'fold {fold+1}/{cfg.nfolds}...')
        fold_start_time = time.time()

        # split data
        x_train, x_val = x.iloc[train_idx], x.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        # define and fit model
        model = lgb.LGBMRegressor(**params)
        model.fit(
            x_train,
            y_train,
            eval_set = [(x_val, y_val)],
            callbacks = [lgb.log_evaluation(100)],
            eval_metric = ['root_mean_squared_error', 'mean_absolute_error'],
        )

        # validation score
        cv_score = np.sqrt(mean_squared_error(y_val, model.predict(x_val)))
        cv_scores.append(cv_score)

        # predict test data
        preds.append(model.predict(xtest))

        fold_run_time = time.time() - fold_start_time
        print(f'rmse: {cv_score:.4f}, run time: {fold_run_time:.2f}\n')

    # print results
    run_time = time.time() - start_time
    print(f'\nTraining completed. Total run time: {run_time:2f}')
    print(f'CV score:\n\t mean: {np.mean(cv_scores):0.6f}\n\t std: {np.std(cv_scores):0.6f}')

    return (cv_scores, preds)

In [13]:
# train model
scores, preds = train_model(train, test, best_params)

****************************** Model: lgb ****************************** 

fold 1/5...
[100]	valid_0's rmse: 0.513631	valid_0's l1: 0.356952
[200]	valid_0's rmse: 0.504782	valid_0's l1: 0.347799
[300]	valid_0's rmse: 0.502972	valid_0's l1: 0.345662
rmse: 0.5030, run time: 3.43

fold 2/5...
[100]	valid_0's rmse: 0.529767	valid_0's l1: 0.367806
[200]	valid_0's rmse: 0.522545	valid_0's l1: 0.360505
[300]	valid_0's rmse: 0.520601	valid_0's l1: 0.358735
rmse: 0.5206, run time: 3.61

fold 3/5...
[100]	valid_0's rmse: 0.523434	valid_0's l1: 0.361468
[200]	valid_0's rmse: 0.514362	valid_0's l1: 0.352955
[300]	valid_0's rmse: 0.511231	valid_0's l1: 0.350155
rmse: 0.5106, run time: 3.99

fold 4/5...
[100]	valid_0's rmse: 0.541401	valid_0's l1: 0.371478
[200]	valid_0's rmse: 0.532976	valid_0's l1: 0.362968
rmse: 0.5304, run time: 3.32

fold 5/5...
[100]	valid_0's rmse: 0.52664	valid_0's l1: 0.364155
[200]	valid_0's rmse: 0.5187	valid_0's l1: 0.356413
[300]	valid_0's rmse: 0.516342	valid_0's l1: 0

In [14]:
# get cv score and final predictions
score = np.mean(scores)
final_preds = np.mean(np.array(preds), axis=0)

In [15]:
# save predictions
sub = pd.read_csv('../data/raw/sample_submission.csv', index_col=0)
sub[TARGET] = final_preds
sub.head()

out_path = '../submissions/'
os.makedirs(out_path, exist_ok=True)
sub.to_csv(out_path + f'{cfg.modelname}.csv')